# Demo of application using lume-epics and lume-model 

In [ ]:
import numpy as np
from lume_epics.epics_server import Server
from lume_epics.model import SurrogateModel

from lume_model.variables import ScalarInputVariable, ImageOutputVariable

## Model class
The model class is as subclass of the lume_epics.model.SurrogateModel class, which will validate that the subclasses have assigned compatible input/output variables and a predict method.

In [ ]:
class DemoModel(SurrogateModel):
    input_variables = {
        "input1": ScalarInputVariable(name="input1", value=1, range=[0, 256]),
        "input2": ScalarInputVariable(name="input2", value=2, range=[0, 256]),
    }

    output_variables = {
        "output1": ImageOutputVariable(
            name="output1", axis_labels=["value_1", "value_2"], axis_units=["mm", "mm"], x_min=0, x_max=50, y_min=0, y_max=50
        )
    }
    
    def evaluate(self, input_variables):
        self.output_variables["output1"].value = np.random.uniform(
            self.input_variables["input1"].value, # lower dist bound
            self.input_variables["input2"].value, # upper dist bound
            (50,50)
        ) #shape
        

        return list(self.output_variables.values())

# Server
Setting up Channel Access and PVAccess servers requires passing only prefix and the model class. Single protocol servers can be configured by passing `protocol=[{PROTOCOL}]`.

In [ ]:
prefix = "test"
server = Server(DemoModel, prefix)
# monitor = False does not loop in main thread
server.start(monitor=False)

# Set up client application
The widgets included in the lume-epics use the epics controller to surface process variables with bokeh.

In [ ]:
from lume_epics.client.widgets.plots import ImagePlot
from lume_epics.client.widgets.sliders import build_sliders
from lume_epics.client.controller import Controller

from bokeh.io import output_notebook, show
from bokeh import palettes
from bokeh.layouts import column, row
from bokeh.models import LinearColorMapper
# load bokeh
output_notebook()

### Build slider and image widgets

In [ ]:
 # build sliders for the command process variable database
inputs = list(DemoModel.input_variables.values())
controller = Controller("pva") # can also use channel access
sliders = build_sliders(inputs, controller, prefix)

# create image plot
output_variables = list(DemoModel.output_variables.values())
image_plot = ImagePlot(output_variables, controller, prefix)

pal = palettes.viridis(256)
color_mapper = LinearColorMapper(palette=pal, low=0, high=256)

image_plot.build_plot(color_mapper=color_mapper)

# Set up image update callback
def image_update_callback():
    image_plot.update()

# Render application 

In [ ]:
# function for rendering the application in the bokeh server
def render_app(doc):
    doc.title = "Demo App"
    doc.add_root(
        column(
            row(column(sliders, width=350)), 
            row(image_plot.plot, height=300),
        )
    )
    doc.add_periodic_callback(image_update_callback, 250)

    
show(render_app)

# Stop the server

In [ ]:
server.stop()